In [6]:
#PUBLIC
import random
import pickle
import numpy as np
import pandas as pd
import numpy.matlib 

from sklearn import svm
from numpy import genfromtxt
from pandas import Series,DataFrame
from sklearn.metrics import accuracy_score 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support


leukemia_big_dataset = pd.read_csv("leukemia_big.csv")
my_data = genfromtxt('leukemia_big.csv', delimiter=',')
leukemia_big_dataset_t = np.transpose(np.flipud(my_data))
df = pd.DataFrame(leukemia_big_dataset_t)
df.to_csv("leukemia_big_22.csv")
leukemia_big_dataset = pd.read_csv("leukemia_big_22.csv")

f = open('leukemia_big_22.csv','r')
l = f.readlines()[1:]
f.close()

All_1 = [e for e in l if e.strip().split(',')[-1] == '1.0']
All_0 = [e for e in l if e.strip().split(',')[-1] == '0.0']

def CretaeTrainTestDev(All_1,All_0,TrainPercent,TestPercent):
    
    Train_1_Indices = random.sample(All_1,TrainPercent*len(All_1)//100)
    Train_0_Indices = random.sample(All_0,TrainPercent*len(All_0)//100)
    Train = Train_0_Indices + Train_1_Indices
    random.shuffle(Train);
    All_1 = [e for e in All_1 if e not in Train_1_Indices]
    All_0 = [e for e in All_0 if e not in Train_0_Indices]
    Test_1_Indices = random.sample(All_1,TestPercent*len(All_1)//100)
    Test_0_Indices = random.sample(All_0,TestPercent*len(All_0)//100)
    Test = Test_0_Indices + Test_1_Indices
    random.shuffle(Test);
    dev_1 = [e for e in All_1 if e not in Test_1_Indices]
    dev_0 = [e for e in All_0 if e not in Test_0_Indices]
    Dev = dev_1 + dev_0
    random.shuffle(Dev)
    print(type(Train),type(Test),type(Dev))
    return Train, Test, Dev
Train,Dev, Test = CretaeTrainTestDev(All_1,All_0,70,20)

o1 = open("Train70.csv",'w')
o2 = open("Test20.csv",'w')
o3 = open("Dev10.csv",'w')
for e in Train:

    o1.write(','.join(e.split(',')[1:]))
for e in Test:
    o2.write(','.join(e.split(',')[1:]))
for e in Dev:
    o3.write(','.join(e.split(',')[1:]))
o1.close()
o2.close()
o3.close()
a = pd.read_csv("Train70.csv")
X_train = a.iloc[:,:-1].values  
y_train = a.iloc[:,-1].values   


a1 = pd.read_csv("Test20.csv")
X_test = a1.iloc[:,:-1].values  
y_test = a1.iloc[:,-1].values   

a2 = pd.read_csv("Dev10.csv")
X_Dev = a2.iloc[:,:-1].values  
y_Dev = a2.iloc[:,-1].values   
print('********************************')
print ('ALL=1','/  AML=0')
print('********************************')

print('X_train shape:{}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))

print('X_validate shape:{}'.format(X_Dev.shape))
print('y_validate shape: {}'.format(y_Dev.shape))

print('X_test shape: {}'.format(X_test.shape))
print('y_test shape: {}'.format(y_test.shape))

#SVM

custom_svm = svm.SVC(kernel='rbf', gamma=0.7, C = 1) 
custom_svm.fit(X_train, y_train)  
pred_svm = custom_svm.predict(X_Dev) 
cm_custom_svm = confusion_matrix(y_Dev,pred_svm)
precisions_svm = cross_val_score(custom_svm, X_train, y_train, cv=10, scoring='precision')
p_svm=np.mean(precisions_svm) 
recalls_svm = cross_val_score(custom_svm, X_train, y_train, cv=5, scoring='recall')
r_svm=np.mean(recalls_svm)
F_svm = 2*p_svm*r_svm/(p_svm+r_svm)
print('p_svm = Precision_svm  = TP_svm / (TP_svm + FP_svm) =', p_svm)
print('----------------------------------')
print('r_svm = Recalls_svm =TP Rate_svm = TP_svm / (TP_svm + FN_svm) = ', r_svm)
print('----------------------------------')
print('F_svm = F.meauser_svm= 2*p_svm*r_svm/(p_svm+r_svm) = ',F_svm)

#BAYES

custom_bay =GaussianNB()
custom_bay.fit(X_train, y_train) 
pred_bay = custom_bay.predict(X_Dev)
cm_custom_bay = confusion_matrix(y_Dev,pred_bay)   
print('----------------------------------')
precisions_bay = cross_val_score(custom_bay, X_train, y_train, cv=10, scoring='precision')
p_bay=np.mean(precisions_bay) 
recalls_bay = cross_val_score(custom_bay, X_train, y_train, cv=5, scoring='recall')
r_bay=np.mean(recalls_bay)
F_bay = 2*p_bay*r_bay/(p_bay+r_bay)
print('p_bay = Precision_bay  = TP_bay / (TP_bay + FP_bay) =', p_bay)
print('----------------------------------')
print('r_bay = Recalls_bay =TP Rate_bay = TP_bay / (TP_bay + FN_bay) = ', r_bay)
print('----------------------------------')
print('F_bay = F.meauser_bay= 2*p_bay*r_bay/(p_bay+r_bay) = ',F_bay)
print('----------------------------------')

#ExtraTreesClassifier

custom_clf = ExtraTreesClassifier() 
custom_clf.fit(X_train, y_train)  
pred_clf = custom_clf.predict(X_Dev) 
cm_custom_clf = confusion_matrix(y_Dev,pred_clf)
print('----------------------------------')
print('----------------------------------')
precisions_clf = cross_val_score(custom_clf, X_train, y_train, cv=10, scoring='precision')
p_clf=np.mean(precisions_clf)
recalls_clf = cross_val_score(custom_clf, X_train, y_train, cv=5, scoring='recall')
r_clf=np.mean(recalls_clf)
F_clf = 2*p_clf*r_clf/(p_clf+r_clf)
print('p_clf = Precision_clf  = TP_clf / (TP_clf + FP_clf) =', p_clf)
print('----------------------------------')
print('r_clf = Recalls_clf =TP Rate_clf = TP_clf / (TP_clf + FN_clf) = ', r_clf)
print('----------------------------------')
print('F_clf = F.meauser_clf= 2*p_clf*r_clf/(p_clf+r_clf) = ',F_clf)
print('----------------------------------')

#CERAT 100 TRAIN

scaller = StandardScaler()
TRAIN_X=np.zeros((100,48,7128))
TRAIN_Y=np.zeros((100,48,1))
i=0
idx=0
for idx in range(100):
    for i in range(48):
        index=random.randint(0,47)
        #print('idx=',idx)
        TRAIN_X[idx,i,:]= X_train[index,:]
        TRAIN_Y[idx,i,:]= y_train[index]

#CREAT 100 MODLE & EVALUATION & SAVE RESULT

j=0
allModels=[] # [(fscore,model),...,(fscore, model)]
Fscore=[]
for j in range(0, 100):   

    custom_bay =GaussianNB()
    model_bay=custom_bay.fit(TRAIN_X[j,:,:], TRAIN_Y[j,:,:])  
    
    filename = 'bay_model_%d.pkl'%(j,)
    z=pickle.dump(model_bay, open(filename,'wb'))
    z=pickle.load(open(filename,'rb'))
    y_pred = z.predict(X_test)
    fscore = precision_recall_fscore_support(y_test,y_pred, average='micro')[2]
    allModels.append((fscore, model_bay))
    Fscore.append(fscore)

# Combine models

def modelVoting(models,X_test, y_test):
    y_pred_all=[]
    for model_tuple in models:
        model = model_tuple[1]
        y_pred = model.predict(X_Dev).tolist()
        y_pred_all.append(y_pred)
    y_pred_all = np.asarray(y_pred_all)
    print(len(y_pred_all))
    y_pred_final = np.zeros(y_Dev.shape)  
    print(y_Dev.shape[0])
    for i in range(y_Dev.shape[0]):
        (value, counts) = np.unique(y_pred_all[:,i], return_counts=True)
        ind = np.argmax(counts)
        y_pred_final[i] = value[ind]
        
    return y_pred_final





allModels=[]
for j in range(0, 100):
    custom_bay =GaussianNB()
    model_bay=custom_bay.fit(TRAIN_X[j,:,:], TRAIN_Y[j,:,:])  
    y_pred = model_bay.predict(X_Dev)
    fscore = precision_recall_fscore_support(y_Dev,y_pred, average='micro')[2]
    allModels.append((fscore, model_bay))
def getKey(item):
    return item[0]
allModels.sort(key=getKey, reverse = True)



# Forward Selection 

allModels.sort(key=getKey, reverse = True)
sl = allModels
sc=[sl[0][1]]
best_score=sl[0][0]
for i in range(1,100):
    temp_list=sc+[sl[i][1]]
    
    voting_y_pred=modelVoting([temp_list], X_Dev, y_Dev)
    
    best_score_temp=precision_recall_fscore_support(y_Dev,voting_y_pred, average='micro')[2]
    if best_score_temp>best_score:
        best_score=best_score_temp
        sc=temp_list[i]
        
print("Results of Forwarding",voting_y_pred)


#Back Ward Elimnation

bestModel = []
allModelsTemp = allModels.copy()

bestModel.append(allModelsTemp[0])

allModelsTemp.pop(0)

best_score = bestModel[0][0] # index 0 is fscore index 1 is model (score, model)
stop = False
while(len(allModelsTemp)>1 and stop == False):
    best_score_temp = -1
    best_index_temp = -1
    k = -1
    for model in allModelsTemp:  
        k += 1
        voting_y_pred = modelVoting(bestModel + [model], X_Dev, y_Dev)
        stop = True
    fscore = precision_recall_fscore_support(y_Dev,voting_y_pred, average='micro')[2]
    best_temp_score = fscore
    fscore
#i=0
while i< (len(allModelsTemp)):
    allModelsTemp.pop(i)
    fscore = precision_recall_fscore_support(y_Dev,voting_y_pred, average='micro')[2]
    if(fscore > best_temp_score):
           best_temp_score = fscore
    else:
        voting_y_pred = modelVoting(bestModel + [model], X_Dev, y_Dev)
    i+=1     
y_pred_backwarding = modelVoting(bestModel, X_Dev, y_Dev)
print("Results of backward elimination",y_pred_backwarding)


#GA ALGORITM

# POPULATION_SIZE = 300
# NUMB_OF_ELITE_CHROMOSOMES = 1
# ROULETTE_WHEEL_SIAZE = 6
# MUTATION_RATE = 0.01
# CROSSOVER_RATE = 0.45
# ITER=5
# i=0

# TARGET_CHROMOSOME=np.zeros(10)# بعدا 100 بزار
# while i < TARGET_CHROMOSOME.__len__() :
#     if random.random () >=0.5 :
#         TARGET_CHROMOSOME[i]= 0
#     else:
#         TARGET_CHROMOSOME[i]= 1
#     i+=1
# print(TARGET_CHROMOSOME)
    
# class chromosome:
#     def __init__ (self) :
#         self._genes=[]
#         #self._fitness =np.zeros((self._genes.__len__()))
#         self._fitness=0
#         i=0
#         while i<TARGET_CHROMOSOME.__len__():
#             if random.random () >= 0.5 :
#                 self._genes.append(1)
#             else:
#                 self._genes.append(0)
#             i+=1
#     def get_genes (self) :
#         return self._genes
    
#     def get_fitness (self) :
         
#         for i in range(self._genes.__len__()):
#             if Fscore [i] >= Fscore[(i+1)]:
#             #if self._genes[i]==TARGET_CHROMOSOME[i]:#  تا جایی ادامه بده که طول ژن انداره کرومورم بشود
#                 #self._fitness += 1
#                 self._fitness = Fscore[i]
#             else :
#                 self._fitness = Fscore[(i+1)]
            
#         return self._fitness
#     def __str__ (self) :
#         return self._genes.__str__()
            

        
# class population:
#     def __init__ (self,size) :
#         self._chromosomesm = []
#         i = 0
#         while i < size:
#             self._chromosomesm.append(chromosome())
#             i += 1
        
#     def get_chromosomes(self): return self._chromosomesm
    
    
    
    
# class GeneticAlgorithm:
#     @staticmethod
#     def evolve(pop) :
#         return GeneticAlgorithm._mutate_population (GeneticAlgorithm._crossover_population(pop))
#     @staticmethod
#     def _crossover_population(pop) :
#         crossover_pop = population(0) #taghir
#         for i in range (NUMB_OF_ELITE_CHROMOSOMES):
#             crossover_pop.get_chromosomes().append(pop.get_chromosomes()[i])
#         while i< POPULATION_SIZE :
#                 chromosome1 = GeneticAlgorithm._select_roulette_population(pop).get_chromosomes()[0]
#                 chromosome2 = GeneticAlgorithm._select_roulette_population(pop).get_chromosomes()[0]
#                 crossover_pop.get_chromosomes().append(GeneticAlgorithm._crossover_chromosomes(chromosome1,chromosome2))
#                 i += 1
#         return crossover_pop
    
#     @staticmethod
#     def _mutate_population(pop) :
#         for i in range (NUMB_OF_ELITE_CHROMOSOMES,POPULATION_SIZE):
#             GeneticAlgorithm._mutate_chromosomes(pop.get_chromosomes()[i])
#         return pop
    
    
    
    
#     @staticmethod
#     def _crossover_chromosomes(chromosome1,chromosome2):
#         crossover_chrom = chromosome()
#         for i in range (TARGET_CHROMOSOME.__len__()):
#             if random.random() >= CROSSOVER_RATE:
#                 crossover_chrom.get_genes()[i] = chromosome1.get_genes()[i]
#             else :
#                 crossover_chrom.get_genes()[i] = chromosome2.get_genes()[i]
#         return crossover_chrom
    
#     @staticmethod
#     def _mutate_chromosomes (chromosome):
#         for i in range(TARGET_CHROMOSOME.__len__()):
#             if random.random() < MUTATION_RATE:
#                 if random.random() < 0.5:
#                     chromosome.get_genes()[i] = 1
#                 else:
#                     chromosome.get_genes()[i] = 0
            
    
#     @staticmethod
#     def _roulette_select_population(pop):
        
#         roulette_pop = population(0)
#         i = 0
#         while i < ROULETTE_WHEEL_SIAZE:
#             roulette_pop.get_chromosomes().append(pop.get_chromosomes()[random.randint(0,POPULATION_SIZE-1)]) #taghir
#             i += 1
#             roulette_pop.get_chromosomes().sort(key=getKey x: x.get_fitness(),reverse=True)
#         return roulette_pop
        
        


#     def _print_population(pop, gen_number): 
#         print("\n---------------------------------------------------------------------")
#         print ("Generation #" , gen_number, "| Fittest chromosome fitness:", pop.get_chromosomes ()[0].get_fitness())
#         print ("Target Chromosome:", TARGET_CHROMOSOME) 
#         print("\n---------------------------------------------------------------------")
#         i = 0
#         for x in pop.get_chromosomes() :
#             print("Chromosome #", i, ":", x, "| Fittest: ", x.get_fitness())
#             i += 1
# #taghir
# populationObj = population(POPULATION_SIZE)
# populationObj.get_chromosomes() .sort(key=getKey x: x.get_fitness(Fscore),reverse=True)
# geneticAlgorithmObj = GeneticAlgorithm
# geneticAlgorithmObj._print_population(populationObj, 0)
# generation_number = 1
# while generation_number < ITER :
#     populationObj = geneticAlgorithmObj.evolve(populationObj)
#     populationObj.get_chromosomes().sort(key=getKey x: x.get_fitness(Fscore),reverse=True)
#     geneticAlgorithmObj._print_population (populationObj, generation_number)
#     generation_number +=1
        
  

# SELCT BEST 
# for i in range(0,4):
#     final_result[0][i]=GA[i]

# for i in range(0,4):
#     final_result[1][i]=FORWARD[i]

# for i in range(0,4):
#     final_result[2][i]=BACKWARD[i]
    
# for i in range(0,4):
#     final_result[3][i]=ALL[i]
    
# FScore_mean=final_result.mean(1)

# print(final_result)
# print(FScore_mean)
# best=0
# for i in range(0,4):
#     if FScore_mean[i]>best :
#         best=FScore_mean[i]
#         idx=i
# if idx==0 :
#     print("GA Best")
# if idx==1:
#     print("FORWARD Best")
# if idx==2 :
#     print("BACKWARD Best")
# if idx==3 :
#     print("ALL Best")


<class 'list'> <class 'list'> <class 'list'>
********************************
ALL=1 /  AML=0
********************************
X_train shape:(48, 7128)
y_train shape: (48,)
X_validate shape:(3, 7128)
y_validate shape: (3,)
X_test shape: (18, 7128)
y_test shape: (18,)
p_svm = Precision_svm  = TP_svm / (TP_svm + FP_svm) = 0.6516666666666666
----------------------------------
r_svm = Recalls_svm =TP Rate_svm = TP_svm / (TP_svm + FN_svm) =  1.0
----------------------------------
F_svm = F.meauser_svm= 2*p_svm*r_svm/(p_svm+r_svm) =  0.7891019172552977
----------------------------------
p_bay = Precision_bay  = TP_bay / (TP_bay + FP_bay) = 0.9550000000000001
----------------------------------
r_bay = Recalls_bay =TP Rate_bay = TP_bay / (TP_bay + FN_bay) =  1.0
----------------------------------
F_bay = F.meauser_bay= 2*p_bay*r_bay/(p_bay+r_bay) =  0.9769820971867008
----------------------------------
----------------------------------
----------------------------------


C:\Users\maryam\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


p_clf = Precision_clf  = TP_clf / (TP_clf + FP_clf) = 0.86
----------------------------------
r_clf = Recalls_clf =TP Rate_clf = TP_clf / (TP_clf + FN_clf) =  0.9047619047619048
----------------------------------
F_clf = F.meauser_clf= 2*p_clf*r_clf/(p_clf+r_clf) =  0.8818132757690231
----------------------------------


C:\Users\maryam\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maryam\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maryam\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maryam\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed w

1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
Results of Forwarding [1. 1. 0.]
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
1
3
Results of backward elimination [1. 1. 1.]
